https://data.mendeley.com/datasets/cyhchpxwps/2

In [3]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features_norm, labels, test_size = 0.2, random_state = 42, stratify = labels
)


file_names = [
    "EEG.xlsx",
    "Linear ECG.xlsx",
    "Nonlinear ECG.xlsx",
    "Ratio of Alpha _ Beta Power.xlsx"
]

control_dir = "/Users/hannahmanheimer/Downloads/Dataset/Control - no negative feedback"
experiment_dir = "/Users/hannahmanheimer/Downloads/Dataset/Experiment - with negative feedback"

def load_data_from_folder(folder, file_names):
    data_frames = []
    for file in file_names:
        file_path = os.path.join(folder, file)
        #needed to debug
        print("Attempting to load:", file_path)
        df = pd.read_excel(file_path)
        prefix = os.path.splitext(file)[0]
        df = df.add_prefix(prefix + "_")
        data_frames.append(df)
    data = pd.concat(data_frames, axis = 1)
    return data


#load the data
control_data = load_data_from_folder(control_dir, file_names)
experiment_data = load_data_from_folder(experiment_dir, file_names)

#combine the datasets
#use 0 and 1 to consider for the negative feedback and no negative feedback
control_data["label"] = 0
experiment_data["label"] = 1

#combine datasets into one
full_data = pd.concat([control_data, experiment_data], axis=0).reset_index(drop=True)
print("Combined data shape:", full_data.shape)

#preprocess
features = full_data.drop("label", axis = 1).select_dtypes(include=[np.number]).values
labels = full_data["label"].values

# Normalize with min-max scaling
feat_min = features.min(axis = 0)
feat_max = features.max(axis = 0)
features_norm = (features - feat_min) / (feat_max - feat_min + 1e-8)


#use pytorch
class StressDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.long)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = StressDataset(X_train, y_train)
test_dataset  = StressDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

#define simple neural network classifier
class StressClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=32, num_classes=2):
        super(StressClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = X_train.shape[1]
model = StressClassifier(input_dim = input_dim, hidden_dim = 32, num_classes = 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_X.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    
    
    #test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, preds = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (preds == batch_y).sum().item()
    acc = correct / total * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Test Accuracy: {acc:.2f}%")

#final model eval
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

final_acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
print("Final Test Accuracy: {:.2f}%".format(final_acc))


Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Control - no negative feedback/EEG.xlsx
Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Control - no negative feedback/Linear ECG.xlsx
Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Control - no negative feedback/Nonlinear ECG.xlsx
Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Control - no negative feedback/Ratio of Alpha _ Beta Power.xlsx
Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Experiment - with negative feedback/EEG.xlsx
Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Experiment - with negative feedback/Linear ECG.xlsx
Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Experiment - with negative feedback/Nonlinear ECG.xlsx
Attempting to load: /Users/hannahmanheimer/Downloads/Dataset/Experiment - with negative feedback/Ratio of Alpha _ Beta Power.xlsx
Combined data shape: (662, 94)
Epoch 1/30, Loss: nan, Test Accuracy: 50.38%
Epoch 2/30, Lo